In [5]:
import requests
import pyodbc
import time
from datetime import datetime
import urllib.parse

In [6]:
API_KEY = "RGAPI-26391bac-5f98-404d-8224-7a9038a32e90"
HEADERS = {"X-Riot-Token": API_KEY}

GAME_NAME = "princethoth"
TAGLINE = "4921"

encoded_name = urllib.parse.quote(GAME_NAME)
encoded_tag = urllib.parse.quote(TAGLINE)

url = f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_name}/{encoded_tag}"

res = requests.get(url, headers=HEADERS)
print(res.status_code)
res.json()

200


{'puuid': 'qnFAHiMjD_NIYvLechb6XYgBvAKYuaYR7lQlVg7U-z_H6F6ndQo1Tr33Eq_1-aqecI0kn0QVuGOU-Q',
 'gameName': 'princethoth',
 'tagLine': '4921'}

In [7]:
PUUID = res.json()["puuid"]

url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{PUUID}"
res = requests.get(url, headers=HEADERS)

print(res.status_code)
res.json()

200


{'puuid': 'qnFAHiMjD_NIYvLechb6XYgBvAKYuaYR7lQlVg7U-z_H6F6ndQo1Tr33Eq_1-aqecI0kn0QVuGOU-Q',
 'profileIconId': 29,
 'revisionDate': 1765721937000,
 'summonerLevel': 1}

In [13]:
PUUID = "qnFAHiMjD_NIYvLechb6XYgBvAKYuaYR7lQlVg7U-z_H6F6ndQo1Tr33Eq_1-aqecI0kn0QVuGOU-Q"

url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{PUUID}/ids"
params = {
    "start": 0,
    "count": 5   # just 5 matches for pilot
}

res = requests.get(url, headers=HEADERS, params=params)
print(res.status_code)
match_ids = res.json()
match_ids

200


['EUW1_7643548973']

In [14]:
match_id = "EUW1_7643548973"

url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"
res = requests.get(url, headers=HEADERS)

print(res.status_code)
match_data = res.json()

match_data.keys()

200


dict_keys(['metadata', 'info'])

In [15]:
participants = match_data["info"]["participants"]

len(participants)

10

In [16]:
participants[0].keys()

dict_keys(['PlayerScore0', 'PlayerScore1', 'PlayerScore10', 'PlayerScore11', 'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5', 'PlayerScore6', 'PlayerScore7', 'PlayerScore8', 'PlayerScore9', 'allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToEpicMonsters', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'it

In [ ]:
# INSERT INTO THE MATCHES TABLE

In [18]:
insert_match_sql = """
INSERT INTO matches (
    match_id,
    match_date,
    duration,
    game_mode,
    map_name
)
VALUES (?, ?, ?, ?, ?)
"""

cursor.execute(
    insert_match_sql,
    metadata["matchId"],
    datetime.fromtimestamp(match_info["gameStartTimestamp"] / 1000),
    match_info["gameDuration"],
    match_info["gameMode"],
    "Summoner's Rift"
)

conn.commit()

In [ ]:
# INSERT INTO PLAYERS TABLE

In [21]:
insert_player_sql = """
IF NOT EXISTS (
    SELECT 1 FROM players WHERE player_id = ?
)
INSERT INTO players (
    player_id,
    region,
    join_date,
    last_active_date,
    level,
    platform
)
VALUES (?, ?, ?, ?, ?, ?)
"""

match_date = datetime.fromtimestamp(match_info["gameStartTimestamp"] / 1000)

for p in participants:
    cursor.execute(
        insert_player_sql,
        p["puuid"],
        p["puuid"],
        "EUW",
        match_date,
        match_date,
        p["summonerLevel"],
        "PC"
    )

conn.commit()

In [ ]:
# INSERT INTO MATCH PARTICIPANTS TABLE

In [ ]:
insert_participant_sql = """
INSERT INTO match_participants (
    match_id,
    player_id,
    champion,
    team,
    kills,
    deaths,
    assists,
    win_flag
)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

for p in participants:
    cursor.execute(
        insert_participant_sql,
        metadata["matchId"],
        p["puuid"],
        p["championName"],
        p["teamId"],
        p["kills"],
        p["deaths"],
        p["assists"],
        1 if p["win"] else 0
    )

conn.commit()

In [ ]:
#CONNECTS TO SQL DATABASE

In [3]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=THOTH\SQLEXPRESS;"
    "DATABASE=Gaming_Product_Analytics;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

In [23]:
#SCALE THE DATASET

In [26]:
seed_players = [
    p["puuid"]
    for p in participants
    if p["puuid"] != "BOT"
]

seed_players

['3Co7YKkCDiwucb-_0xVWyjdQOh2mPOwCfBj7eSe91i0dnjp9owtn7vArHp0JQnxrX_5C3zILGbD93g',
 'aJ750DxT1LhERbv73_4mAlNxsT3yOpi2ZNoP_Ut3DoJ7-uM_K-QH4ZF_c3aOmvEj9Q2mdE3wgaFM5w',
 'rm8NfyDXDgoTuBX6tVojZErtdEE96ULNrkbGwNxajGL8o-keV_HXPejV37181s6UBIYRq7kiGevYDw',
 'RSY44afyLWciPgFOG_imi3UcySzJ4GvpLgE1X811HwSgiD6insDOM68x02wkXMEbMKpxPjAiweuX-g',
 'qnFAHiMjD_NIYvLechb6XYgBvAKYuaYR7lQlVg7U-z_H6F6ndQo1Tr33Eq_1-aqecI0kn0QVuGOU-Q']

In [ ]:
all_match_ids = set()
MAX_MATCHES = 200

for puuid in seed_players:
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {"start": 0, "count": 20}

    res = requests.get(url, headers=HEADERS, params=params)
    res.raise_for_status()

    match_ids = res.json()
    all_match_ids.update(match_ids)

    if len(all_match_ids) >= MAX_MATCHES:
        break

In [28]:
len(all_match_ids)

61

In [29]:
expanded_players = set(seed_players)

for match_id in list(all_match_ids):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()

    data = res.json()
    for p in data["info"]["participants"]:
        expanded_players.add(p["puuid"])

    if len(expanded_players) >= 50:
        break

In [30]:
for puuid in list(expanded_players):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {"start": 0, "count": 20}

    res = requests.get(url, headers=HEADERS, params=params)
    res.raise_for_status()

    all_match_ids.update(res.json())

    if len(all_match_ids) >= MAX_MATCHES:
        break

In [31]:
len(all_match_ids)

200

In [32]:
#INSERT INTO SQL DATABASE

In [33]:
def match_exists(match_id):
    cursor.execute(
        "SELECT 1 FROM matches WHERE match_id = ?", match_id
    )
    return cursor.fetchone() is not None

In [35]:
inserted = 0

for match_id in list(all_match_ids):
    if inserted >= MAX_MATCHES:
        break

    if match_exists(match_id):
        continue

    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"
    res = requests.get(url, headers=HEADERS)

    if res.status_code == 429:
        print("Rate limit hit. Sleeping 10 seconds...")
        time.sleep(10)
        continue

    res.raise_for_status()

    data = res.json()
    info = data["info"]
    meta = data["metadata"]
    participants = info["participants"]

    # insert match
    cursor.execute(
        """
        INSERT INTO matches (match_id, match_date, duration, game_mode, map_name)
        VALUES (?, ?, ?, ?, ?)
        """,
        meta["matchId"],
        datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
        info["gameDuration"],
        info["gameMode"],
        "Summoner's Rift"
    )

    # insert players + participants 
    for p in participants:
        cursor.execute(
            """
            IF NOT EXISTS (SELECT 1 FROM players WHERE player_id = ?)
            INSERT INTO players (player_id, region, join_date, last_active_date, level, platform)
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            p["puuid"],
            p["puuid"],
            "EUW",
            datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
            datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
            p["summonerLevel"],
            "PC"
        )

        cursor.execute(
            """
            IF NOT EXISTS (
                SELECT 1 FROM match_participants
                WHERE match_id = ? AND player_id = ?
            )
            INSERT INTO match_participants
            (match_id, player_id, champion, team, kills, deaths, assists, win_flag)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """,
            meta["matchId"],
            p["puuid"],
            meta["matchId"],
            p["puuid"],
            p["championName"],
            p["teamId"],
            p["kills"],
            p["deaths"],
            p["assists"],
            1 if p["win"] else 0
        )

    conn.commit()
    inserted += 1

    time.sleep(1.2)

In [38]:
API_KEY = "RGAPI-a22b1ca5-0cb8-44c7-8150-3132f5e49b9f"
HEADERS = {"X-Riot-Token": API_KEY}

In [39]:
test_puuid = seed_players[0]

url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{test_puuid}/ids"
params = {"start": 0, "count": 5}

res = requests.get(url, headers=HEADERS, params=params)
print(res.status_code)
print(res.json())

200
['EUW1_7643655247', 'EUW1_7643604812', 'EUW1_7643548973', 'EUW1_7643326750', 'EUW1_7642899820']


In [42]:
all_match_ids = set()
MAX_MATCHES = 5000

for puuid in seed_players:
    start = 0

    while True:
        url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
        params = {"start": start, "count": 100}

        res = requests.get(url, headers=HEADERS, params=params)
        res.raise_for_status()

        match_ids = res.json()

        if not match_ids:
            break

        all_match_ids.update(match_ids)

        if len(all_match_ids) >= MAX_MATCHES:
            break

        start += 100
        time.sleep(1.2)

    if len(all_match_ids) >= MAX_MATCHES:
        break

In [43]:
len(all_match_ids)

1805

In [52]:
len(all_players)

403

In [53]:
inserted = 0

for match_id in list(all_match_ids):
    if inserted >= MAX_MATCHES:
        break

    if match_exists(match_id):
        continue

    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"
    res = requests.get(url, headers=HEADERS)

    if res.status_code == 429:
        print("Rate limit hit. Sleeping 10 seconds...")
        time.sleep(10)
        continue

    res.raise_for_status()

    data = res.json()
    info = data["info"]
    meta = data["metadata"]
    participants = info["participants"]

    # insert match
    cursor.execute(
        """
        INSERT INTO matches (match_id, match_date, duration, game_mode, map_name)
        VALUES (?, ?, ?, ?, ?)
        """,
        meta["matchId"],
        datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
        info["gameDuration"],
        info["gameMode"],
        "Summoner's Rift"
    )

    # insert players + participants 
    for p in participants:
        cursor.execute(
            """
            IF NOT EXISTS (SELECT 1 FROM players WHERE player_id = ?)
            INSERT INTO players (player_id, region, join_date, last_active_date, level, platform)
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            p["puuid"],
            p["puuid"],
            "EUW",
            datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
            datetime.fromtimestamp(info["gameStartTimestamp"] / 1000),
            p["summonerLevel"],
            "PC"
        )

        cursor.execute(
            """
            IF NOT EXISTS (
                SELECT 1 FROM match_participants
                WHERE match_id = ? AND player_id = ?
            )
            INSERT INTO match_participants
            (match_id, player_id, champion, team, kills, deaths, assists, win_flag)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """,
            meta["matchId"],
            p["puuid"],
            meta["matchId"],
            p["puuid"],
            p["championName"],
            p["teamId"],
            p["kills"],
            p["deaths"],
            p["assists"],
            1 if p["win"] else 0
        )

    conn.commit()
    inserted += 1

    time.sleep(1.2)